Prima di fare partire il notebook, installa l'ultimo firefox disponibile
e scarica anche geckodriver da qui > https://github.com/mozilla/geckodriver/releases, estrai e dai il path qui sotto, crea anche una cartella dove salvare i json, modifica qui sotto il link sulla mia cartella sul desktop

In [ ]:
geckodriver_path = r"C:\Users\ITSIZAM\Desktop\geckodriver-v0.27.0-win64\geckodriver.exe"

In [1]:
%pylab inline
!pip install selenium
!pip install pandas
%cd C:\Users\ITSIZAM\Desktop\Autoscout24_scraping\

Populating the interactive namespace from numpy and matplotlib
C:\Users\ITSIZAM\Desktop\Autoscout24_scraping


In [2]:
!dir

 Volume in drive C is System
 Volume Serial Number is 162F-6FE6

 Directory of C:\Users\ITSIZAM\Desktop\Autoscout24_scraping

09/28/2020  03:23 PM    <DIR>          .
09/28/2020  03:23 PM    <DIR>          ..
09/28/2020  03:14 PM    <DIR>          .ipynb_checkpoints
09/28/2020  02:58 PM         1,086,381 colab_notebook_scraping_jupyter.ipynb
09/28/2020  03:24 PM            35,400 geckodriver.log
09/28/2020  03:19 PM             6,166 ipynb.py
09/28/2020  03:23 PM         1,086,654 notebook_scraping_jupyter.ipynb
09/28/2020  01:08 PM         1,009,012 pagina_di_ricerca_fuel_B_condition_N.json
09/28/2020  02:41 PM         1,117,797 pagina_di_ricerca_fuel_D_condition_N.json
               6 File(s)      4,341,410 bytes
               3 Dir(s)  313,032,732,672 bytes free


In [3]:
#inizializza webdriver
from selenium import webdriver
from tqdm.notebook import tqdm as tqdm

options = webdriver.FirefoxOptions()
options.headless = True

#webdriver di firefox si può scaricare qui > https://github.com/mozilla/geckodriver/releases, estrai e dai il path qui sotto
wd = webdriver.Firefox(options=options, 
                       executable_path= geckodriver_path)
wd.set_window_size(1920,1080)

In [4]:
def loop_offer_pages(x):

  id = x.get_attribute("data-guid") #guid dell'annuncio
  try:
    page = x.find_element_by_css_selector(".cldt-summary-titles > a").get_attribute("href") #pagina dell'annauncio
    vehicle = x.find_element_by_css_selector(".cldt-summary-makemodel.sc-font-bold.sc-ellipsis").text
    price = x.find_elements_by_css_selector("""[data-item-name="price"]""")[0].text # 23 restituisce se c'è una rata, iva dedicibile etc
    vehicle_data = x.find_elements_by_css_selector(".cldt-summary-vehicle-data")[0].text.split("\n")
  except:
    page = ""
    vehicle = ""
    price = ""
    vehicle_data = ""
  try:
    vehicle_usr_desc = x.find_elements_by_css_selector(".cldt-summary-version")[0].text
  except:
     vehicle_usr_desc = ""
  seller_list = x.find_elements_by_css_selector("""[data-item-name="seller"]""")[0].text.split("\n")
  if seller_list[1].find('(') != -1:
    del seller_list[1]
  seller = seller_list[0]
  seller_country = seller_list[1]
  seller_address = seller_list[2]
  return {
      'id_annuncio': id,
      'link_annuncio': page,
      'vehicle': vehicle,
      'vehicle_user_desc' : vehicle_usr_desc,
      'asking_price' : price,
      'seller' : seller,
      'country' : seller_country,
      'address' : seller_address,
      }

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import time

#create blank list to enclose all offers page results
output_scraping = []
tipo_carrozzeria = [1,2,3,4,5,6,7,12]
fuel =  "D" #["B","D","E","L","C","2"]
condizione_auto = "N" #N nuovo, S KM0, U usato

for j in tqdm(tipo_carrozzeria):
  for i in tqdm(range(1,21)): #range to all pages is 21

    url = "https://www.autoscout24.it/lst/?sort=age&" + "offer=" + condizione_auto + "&desc=1&fuel=" + str(fuel) + \
    "&ustate=N%2CU&size=20&page=" + str(i) + \
    "&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=" + str(j) + "&atype=C&fc=14&qry=&"

    print("Pagina: " + str(i) + ", condition: " + condizione_auto + ", fuel: " + str(fuel) + ", tipo carrozzeria: " + str(j) + ", url: " + url)
    #time.sleep(abs(np.random.normal(1,.5)))
    wd.get(url)

    try:
      WebDriverWait(wd, 7).until(
          EC.element_to_be_clickable((By.CSS_SELECTOR, "a.active")) #loads bottom bar, which is the last to be loaded in chromium
      )
    except:
      continue
    
    lista_annunci = wd.find_elements_by_css_selector(".cl-list-element.cl-list-element-gap")

    for x in lista_annunci:
      output_scraping.append(loop_offer_pages(x))
  
    if len(lista_annunci) < 20: #exit if there are less than 20 offers on 1 page
      break

Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 1, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=1&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 1, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=1&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 1, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=1&atype=C&fc=14&qry=&



Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 2, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=2&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 2, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=2&atype=C&fc=14&qry=&



Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 3, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=3&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 3, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=3&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 3, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=3&atype=C&fc=14&qry=&
Pagina: 4, condition: N, fuel: D, tipo carrozzeria: 3, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=4&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=3&atype=C&fc=14&qry=&
Pagina: 5, condition: N, fuel: D, tipo carrozzeria: 3, url: https://www.autoscout24.it/l

Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 4, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=4&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 4, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=4&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 4, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=4&atype=C&fc=14&qry=&
Pagina: 4, condition: N, fuel: D, tipo carrozzeria: 4, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=4&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=4&atype=C&fc=14&qry=&
Pagina: 5, condition: N, fuel: D, tipo carrozzeria: 4, url: https://www.autoscout24.it/l

Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 5, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=5&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 5, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=5&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 5, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=5&atype=C&fc=14&qry=&
Pagina: 4, condition: N, fuel: D, tipo carrozzeria: 5, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=4&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=5&atype=C&fc=14&qry=&
Pagina: 5, condition: N, fuel: D, tipo carrozzeria: 5, url: https://www.autoscout24.it/l

Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 6, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=6&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 6, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=6&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 6, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=6&atype=C&fc=14&qry=&
Pagina: 4, condition: N, fuel: D, tipo carrozzeria: 6, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=4&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=6&atype=C&fc=14&qry=&
Pagina: 5, condition: N, fuel: D, tipo carrozzeria: 6, url: https://www.autoscout24.it/l

Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 7, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=7&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 7, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=7&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 7, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=7&atype=C&fc=14&qry=&
Pagina: 4, condition: N, fuel: D, tipo carrozzeria: 7, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=4&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=7&atype=C&fc=14&qry=&
Pagina: 5, condition: N, fuel: D, tipo carrozzeria: 7, url: https://www.autoscout24.it/l

Pagina: 1, condition: N, fuel: D, tipo carrozzeria: 12, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=1&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=12&atype=C&fc=14&qry=&
Pagina: 2, condition: N, fuel: D, tipo carrozzeria: 12, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=2&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=12&atype=C&fc=14&qry=&
Pagina: 3, condition: N, fuel: D, tipo carrozzeria: 12, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=3&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=12&atype=C&fc=14&qry=&
Pagina: 4, condition: N, fuel: D, tipo carrozzeria: 12, url: https://www.autoscout24.it/lst/?sort=age&offer=N&desc=1&fuel=D&ustate=N%2CU&size=20&page=4&lon=9.18812&lat=45.46362&zip=Milano&zipr=200&cy=I&body=12&atype=C&fc=14&qry=&
Pagina: 5, condition: N, fuel: D, tipo carrozzeria: 12, url: https://www.autosco

In [6]:
print(output_scraping)
print(len(output_scraping))

[{'id_annuncio': '92ee9cae-5fa5-48f0-8c86-322ab2b4604b', 'link_annuncio': 'https://www.autoscout24.it/annunci/ligier-js-50-dci-sport-ultimate-ice-clima-ligier-firenze-diesel-grigio-92ee9cae-5fa5-48f0-8c86-322ab2b4604b?cldtidx=1&cldtsrc=listPage', 'vehicle': 'Ligier JS 50', 'vehicle_user_desc': 'DCI Sport Ultimate Ice CLIMA - Ligier Firenze', 'asking_price': '€ 14.270,- 1', 'seller': 'Alfieri Auto Srl', 'country': 'IT', 'address': '50100 Calenzano - Firenze - Fi'}, {'id_annuncio': 'eb5bb848-f5de-419b-ba86-9ab42ac429c6', 'link_annuncio': 'https://www.autoscout24.it/annunci/aixam-gti-gti-emotion-diesel-grigio-eb5bb848-f5de-419b-ba86-9ab42ac429c6?cldtidx=2&cldtsrc=listPage', 'vehicle': 'Aixam GTI', 'vehicle_user_desc': 'GTI Emotion', 'asking_price': '€ 15.718,- 41', 'seller': 'Parisi 2000 Srl', 'country': 'Resp Vendite', 'address': 'IT'}, {'id_annuncio': '27f89d53-0864-4e7b-910f-0b8c04a65d6b', 'link_annuncio': 'https://www.autoscout24.it/annunci/chatenet-ch-40-st-diesel-nero-27f89d53-0864-

In [7]:
import json
export_output_scraping = open('pagina_di_ricerca_fuel_' + fuel + '_condition_' + condizione_auto + '.json', mode='w+')
export_output_scraping.write(json.dumps(output_scraping, indent=3))
export_output_scraping.close()

In [8]:
wd.quit() #exits firefox driver and resets allocated memory 

options = webdriver.FirefoxOptions()
options.headless = True

#webdriver di firefox si può scaricare qui > https://github.com/mozilla/geckodriver/releases, estrai e dai il path
wd = webdriver.Firefox(options=options, 
                       executable_path= geckodriver_path)
wd.set_window_size(1920,1080)

In [9]:
def loop_single_offer_pages(k):
    url = k['link_annuncio']
    try:
        wd.get(url)

        details_raw = []
        details_value = wd.find_elements_by_css_selector("""[data-item-name="car-details"] >* dd""")
        details_field = wd.find_elements_by_css_selector("""[data-item-name="car-details"] >* dt""")

        for m in range(0,len(details_field)):
            details_raw_field = details_field[m].text
            details_raw_value = details_value[m].text
            if details_raw_value == "":
                details_raw_value = "Sì"
            details_raw.append(details_raw_field)
            details_raw.append(details_raw_value)
        it = iter(details_raw) 
        details = dict(zip(it, it)) 
        details['id_annuncio'] = k['id_annuncio']

        return details
    
    except:
        pass

In [ ]:
output_single_offers_scraping = []
j = 0

for i in tqdm(output_scraping):
    j = j + 1
    
    if j%100 == 0: #exits firefox driver after 100 loops to clear allocated memory
        wd.quit()  
        options = webdriver.FirefoxOptions()
        options.headless = True
        wd = webdriver.Firefox(options=options, 
                       executable_path= geckodriver_path)
        wd.set_window_size(1920,1080)
        
        
    output_single_offers_scraping.append(loop_single_offer_pages(i))

In [ ]:
print (len(output_single_offers_scraping))
print(output_single_offers_scraping)

In [ ]:
#printa .json
export_output_single_offers_scraping = open('singole_pagine_fuel_' + fuel + '_condition_' + condizione_auto + '.json', mode='w+')
export_output_single_offers_scraping.write(json.dumps(output_single_offers_scraping, indent=3))
export_output_single_offers_scraping.close()

In [ ]:
#close webdriver (which is hidden)
wd.Close()